# 간단한 분류 모델 구현하기
**분류(classification)** : 패턴을 파악해 여러 종류로 구분하는 작업
* ex) 어떠한 사진이 고양이잊니, 강아지인지 또는 자동차인지 판단하는 일

In [30]:
# Numpy : 수치해석용 파이썬 라이브러리
import tensorflow as tf
import numpy as np

In [31]:
# [털, 날개] 0: 무, 1: 유 
# 데이터 구성
x_data = np.array([
    [0, 0],
    [1, 0],
    [1, 1],
    [0, 0],
    [0, 0],
    [0, 1]
])

각 개체가 실제 어떤 종류인지를 나타내는 **레이블(분류값) 데이터**를 구성한다.
* **레이블 데이터** : 원-핫 인코딩(one-hot encoding)이라는 특수한 현태로 구성한다.
  * **원-핫 인코딩** : 데이터가 가질 수 있는 값들을 일렬로 나열한 배열을 만들고, 그중 표현하려는 값을 뜻하는 인덱스의 원소만 1로 표기하고 나머지 원소는 모두 9으로 채우는 표기법
  * ex) 기타 = [1, 0, 0], 포유류 = [0, 1, 0], 조류 = [0, 0, 1]

In [32]:
y_data = np.array([
    [1, 0, 0],      # 기타
    [0, 1, 0],      # 포유류
    [0, 0, 1],      # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

**플레이스홀더(placeholder)** : 학습용 데이터를 담는 그릇이다.
* 선언 예시 : **tf.placeholder(dtype, shape, name)**
  * dtype : 플레이스홀더에 저장되는 데이터형
  * shape : 행렬의 차원
  * name : 플레이스홀더의 이름

In [33]:
# 신경망 모델 구성, X와 레이블 Y와의 관계를 알아내는 모델
# X와 Y를 플레이스홀더로 설정한다.
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

**변수형(Variable)** : 가중치와 편향을 만들어주기 위함
* tf.random_uniform : 0~1 사이의 균등확률분포 값을 생성해주는 함수
* tf.zeros : 모든 원소의 값이 0인 텐서를 생성한다.

In [34]:
# 가중치와 편향 변수 설정
# W = [입력층(특징 수), 출력층(레이블 수)]
# b = ([레이블 수])
W = tf.Variable(tf.random_uniform([2, 3], -1., 1.))
b = tf.Variable(tf.zeros([3]))

In [35]:
# 위의 가중치를 곱하고 편향을 더하고 활성화 함수인 ReLU에 적용하면 신경망 구성 완료
L = tf.add(tf.matmul(X, W), b)
L = tf.nn.relu(L)

In [36]:
# 신경망을 통해 나온 출력값을 softmax 함수를 통해 사용하기 쉽게 한다.
model = tf.nn.softmax(L)

In [37]:
# 손실 함수 작성
# 교차 엔트로피 함수 : 예측값과 실제값 사이의 확률 분포 차이를 계산한 값
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis=1))

In [38]:
# 학습
# 기본적인 경사하강법으로 최적화한다.
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

# 텐서플로의 세션을 초기화한다.
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 앞서 구성한 특징과 레이블 데이터를 이용해 학습을 100번 진행한다.
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    # 학습 도중 10번에 한 번씩 손실값 출력
    if (step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict={X:x_data, Y:y_data}))

10 1.123563
20 1.1106182
30 1.0982074
40 1.0863138
50 1.07492
60 1.0640074
70 1.0535575
80 1.0435511
90 1.0339689
100 1.0247921


In [39]:
# 학습 결과 출력
prediction = tf.argmax(model, axis=1)
target = tf.argmax(Y, axis=1)
print("예측값: " , sess.run(prediction, feed_dict={X: x_data}))
print("실제값: " , sess.run(target, feed_dict={Y: y_data}))

예측값:  [2 0 2 2 2 2]
실제값:  [0 1 2 0 0 2]


In [40]:
# 정확도 출력
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도 : %.2f' %sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

정확도 : 33.33


# 심층 신경망 구현하기

In [41]:
# 다층 신경망은 앞서 만든 신경망 모델에 가중치와 편향을 추가하기만 하면 된다.

# 가중치
# W1 = [특징, 은닉층의 뉴런 수]
# W2 = [은닉층의 뉴런 수, 분류 수]
W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
W2 = tf.Variable(tf.random_uniform([10, 3], -1., 1.))

# 편향
# b1 = [은닉층의 뉴런 수]
# b2 = [분류 수]
b1 = tf.Variable(tf.zeros([10]))
b2 = tf.Variable(tf.zeros([3]))

In [42]:
# 가중치와 편향 그리고 활성화 함수를 적용한다.
L1 = tf.add(tf.matmul(X, W1), b1)
L1 = tf.nn.relu(L1)

In [43]:
# 가중치와 편향을 적용하여 최종 모델을 만든다.
model = tf.add(tf.matmul(L1, W2), b2)

In [44]:
# 손실 함수 작성
# 엔트로피 함수를 사용한다.
cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

In [45]:
# 텐서플로의 세션을 초기화한다.
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 앞서 구성한 특징과 레이블 데이터를 이용해 학습을 100번 진행한다.
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    # 학습 도중 10번에 한 번씩 손실값 출력
    if (step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict={X:x_data, Y:y_data}))

10 0.8923652
20 0.7374669
30 0.591422
40 0.464195
50 0.34549627
60 0.25169766
70 0.18228318
80 0.13196768
90 0.09690523
100 0.0697935


In [46]:
# 학습 결과 출력
prediction = tf.argmax(model, axis=1)
target = tf.argmax(Y, axis=1)
print("예측값: " , sess.run(prediction, feed_dict={X: x_data}))
print("실제값: " , sess.run(target, feed_dict={Y: y_data}))

예측값:  [0 1 2 0 0 2]
실제값:  [0 1 2 0 0 2]


In [47]:
# 정확도 출력
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도 : %.2f' %sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

정확도 : 100.00


In [48]:
# 데이터 파일 만들기
sum_data = np.hstack((x_data, y_data))
print(sum_data)

[[0 0 1 0 0]
 [1 0 0 1 0]
 [1 1 0 0 1]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 1 0 0 1]]


In [49]:
import csv

csvfile = open('./data.csv', 'w', newline="")

csvwriter = csv.writer(csvfile)
for row in sum_data:
    csvwriter.writerow(row)
    
csvfile.close()